# Oakland University Baseball Analytics - 2024 Basic counts

Author : Aswini Sivakumar

# Converting the attributes from XML to counts

## Loading all the required libraries

In [ ]:
import os
import re
import pandas as pd
import xml.etree.ElementTree as ET

Load and parse the XML files

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Parse the XML file and get the root of the XML tree

In [ ]:
# Define the directory where the XML files are located
directory = '/content/drive/MyDrive/OU Baseball Stats/XML_Files_2024/'

# Initialize a list to store the root of each XML tree
roots = []

# Iterate over each file in the directory
for filename in os.listdir(directory):
    # Check if the file is an XML file
    if filename.endswith('.XML'):
        # Parse the XML file and get the root of the XML tree
        tree = ET.parse(os.path.join(directory, filename))
        root = tree.getroot()

        # Add the root to the list
        roots.append(root)

Identify the respective tag for the Oakland University team and extract the overall player stats and play-by-play stats for the games played

In [ ]:
# Find Tags where team id is "OAK"
oak_team = []
for root in roots:
    oak_team.extend(root.findall('.//team[@id="OAK"]'))

# Get all the players in "OAK" team
players = []
for team in oak_team:
    players.extend(team.findall('.//player'))

# Get the innings wise batting stats
batting = []
for root in roots:
    batting.extend(root.findall('.//batting[@id="OAK"]'))

# Get play-by-play stats for each innings
play_data = []
for plays in batting:
    play_data.extend(plays.findall('.//play'))


Extract basic counts

In [ ]:
data = []
# Iterate through each player
for player in players:
    player_name = player.attrib['name']
    hsituations = player.findall('hsituation')
    for hsituation in hsituations:
        context = hsituation.attrib['context']
        ab = hsituation.attrib['ab']
        r = hsituation.attrib.get('r', '0')  # Provide '0' as the default value if 'r' is not present
        h = hsituation.attrib.get('h', '0')  # Provide '0' as the default value if 'h' is not present
        rbi = hsituation.attrib.get('rbi', '0')  # Provide '0' as the default value if 'rbi' is not present
        double = hsituation.attrib.get('double', '0')  # Provide '0' as the default value if 'double' is not present
        triple = hsituation.attrib.get('triple', '0')
        hr = hsituation.attrib.get('hr', '0')
        sb = hsituation.attrib.get('sb', '0')  # Provide '0' as the default value if 'sb' is not present
        bb = hsituation.attrib.get('bb', '0')
        so = hsituation.attrib.get('so', '0')
        kl = hsituation.attrib.get('kl', '0')
        hitdp = hsituation.attrib.get('hitdp', '0')
        ground = hsituation.attrib.get('ground', '0')
        fly = hsituation.attrib.get('fly', '0')
        hbp = hsituation.attrib.get('hbp', '0')
        sf = hsituation.attrib.get('sf', '0')
        sh = hsituation.attrib.get('sh', '0')
        cs = hsituation.attrib.get('cs', '0')



        # Extract rcherr from hsitsummary
        hsitsummary = player.find('hsitsummary')
        if context == "bypos":                            # hsitsummary doesn't have context attribute. To avoid grouping error later, adding roe value only for "bypos" context
            roe = hsitsummary.attrib.get('rcherr', '0')  # Provide '0' as the default value if 'rcherr' is not present
        else:
            roe = '0'  # Set to '0' if context is not "bypos"

        fielding = player.find('fielding')
        if context == "bypos":                            # fielding doesn't have context attribute. To avoid grouping error later, adding po value only for "bypos" context
            po = fielding.attrib.get('po', '0')  # Provide '0' as the default value if 'po' is not present
            a = fielding.attrib.get('a','0')
        else:
            po = '0'
            a = '0'

        # Storing all the variables collected above to the data list
        data.append([player_name, context, ab, r, h, rbi, double, triple, hr, sb, bb, so, kl, hitdp, ground, fly, hbp, sf, sh, cs, roe, po, a])

In [ ]:
Oak_hitting = pd.DataFrame(data, columns=['Player Name', 'Context', 'AB', 'R', 'H', 'RBI', 'Double','Triple', 'HR', 'SB', 'BB', 'SO', 'KL', 'HITDP', 'Ground', 'Fly', 'HBP', 'SF', 'SH', 'CS', 'ROE', 'PO', 'A'])

In [ ]:
Final_all = Oak_hitting[(Oak_hitting['Player Name'].isin(['Ian Cleary', 'Reggie Bussey', 'Trent Rice', 'Lucas Day', 'John Lauinger', 'Sam Griffith', 'Brandon Heidal','Drew Gaskins','Aidan Orr','Brandon Nigh','Gavin Arseneau','Erik Larsen','Harrison Griffith','Carter Hain','Ben Clark','Pierson Gunnell','Liam Pollock','Jack Lux'])) & (Oak_hitting['Context'].isin(['bypos','vsleft','vsright','scorepos']))].copy()

In [ ]:
# Converting the mentioned columns to numeric
cols_to_convert = ['AB', 'R', 'H', 'RBI', 'Double', 'Triple', 'HR', 'SB', 'BB', 'SO', 'KL', 'HITDP', 'Ground', 'Fly', 'HBP', 'SF', 'SH' , 'CS' , 'ROE', 'PO', 'A']

for col in cols_to_convert:
    Final_all[col] = pd.to_numeric(Final_all[col], errors='coerce')

Aggregating counts based on player name

In [ ]:
result_new = Final_all.groupby(['Player Name', 'Context']).agg(
    AB=('AB', 'sum'),
    R=('R', 'sum'),
    H=('H', 'sum'),
    RBI=('RBI', 'sum'),
    Double=('Double', 'sum'),
    Triple=('Triple', 'sum'),
    HR=('HR', 'sum'),
    SB=('SB', 'sum'),
    BB=('BB', 'sum'),
    SO=('SO', 'sum'),
    KL=('KL', 'sum'),
    GDP=('HITDP', 'sum'),
    Ground=('Ground', 'sum'),
    Fly=('Fly', 'sum'),
    HBP=('HBP', 'sum') ,
    SF=('SF', 'sum') ,
    SH=('SH', 'sum') ,
    CS=('CS', 'sum') ,
    ROE=('ROE', 'sum'),
    PO=('PO', 'sum'),
    A=('A', 'sum')
).reset_index()
result_new

,Player Name,Context,AB,R,H,RBI,Double,Triple,HR,SB,...,GDP,Ground,Fly,HBP,SF,SH,CS,ROE,PO,A
0,Aidan Orr,bypos,47,10,19,12,5,3,0,2,...,3,15,4,5,0,0,2,0,0,0
1,Aidan Orr,scorepos,12,0,8,11,2,1,0,0,...,0,2,0,3,0,0,0,0,0,0
2,Aidan Orr,vsleft,8,1,3,3,0,2,0,1,...,1,2,1,0,0,0,0,0,0,0
3,Aidan Orr,vsright,39,9,16,9,5,1,0,1,...,2,13,3,5,0,0,2,0,0,0
4,Ben Clark,bypos,7,1,3,2,0,0,1,2,...,0,0,3,0,1,0,0,0,16,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,Sam Griffith,vsright,50,8,12,13,0,0,1,2,...,1,12,13,2,0,1,0,0,0,0
68,Trent Rice,bypos,55,19,17,4,0,2,0,18,...,3,13,9,1,0,1,2,1,31,4
69,Trent Rice,scorepos,19,0,5,3,0,0,0,0,...,0,7,3,0,0,1,0,0,0,0
70,Trent Rice,vsleft,14,5,3,0,0,1,0,8,...,2,4,3,0,0,0,0,0,0,0


In [ ]:
Final_bypos = result_new[result_new['Context'] == 'bypos'].copy()

In [ ]:
# Split the Player name into first name and last name
Final_bypos[['Last Name', 'First Name']] = Final_bypos['Player Name'].str.split(expand=True)

# Combine the columns to get the desired format
Final_bypos['Player Name'] = Final_bypos['Last Name'] + ', ' + Final_bypos['First Name']

# Drop the intermediate columns if needed
Final_bypos.drop(columns=['Last Name', 'First Name'], inplace=True)

# Reorder the columns
Final_bypos = Final_bypos[['Player Name', 'Context', 'AB', 'R', 'H', 'RBI', 'Double', 'Triple', 'HR', 'SB', 'BB', 'SO', 'KL', 'GDP', 'Ground', 'Fly', 'HBP', 'SF', 'SH' , 'CS' , 'ROE', 'PO', 'A']]

Final_bypos

,Player Name,Context,AB,R,H,RBI,Double,Triple,HR,SB,...,GDP,Ground,Fly,HBP,SF,SH,CS,ROE,PO,A
0,"Aidan, Orr",bypos,47,10,19,12,5,3,0,2,...,3,15,4,5,0,0,2,0,0,0
4,"Ben, Clark",bypos,7,1,3,2,0,0,1,2,...,0,0,3,0,1,0,0,0,16,0
8,"Brandon, Heidal",bypos,71,16,22,15,5,2,2,2,...,2,15,17,6,0,0,1,1,102,11
12,"Brandon, Nigh",bypos,74,18,21,23,4,1,3,4,...,2,10,28,5,2,0,3,3,15,70
16,"Carter, Hain",bypos,23,13,12,11,2,0,1,2,...,0,5,3,1,0,1,0,0,21,7
20,"Drew, Gaskins",bypos,23,2,3,2,0,0,1,1,...,1,10,3,1,0,0,0,1,14,17
24,"Erik, Larsen",bypos,30,3,9,4,3,1,1,0,...,0,6,8,0,0,0,0,0,16,23
28,"Gavin, Arseneau",bypos,23,6,4,5,0,0,0,3,...,0,3,9,3,0,0,1,3,21,1
32,"Harrison, Griffith",bypos,7,2,2,5,0,0,1,0,...,0,1,1,1,0,0,0,0,9,0
36,"Ian, Cleary",bypos,58,16,22,13,4,1,1,2,...,1,14,9,2,1,1,0,1,33,1


Calculating Bunts and BNT 1B

In [ ]:
play_df = []

for play in play_data:
    player_name = play.attrib['batter']
    pchprof = play.attrib['pchprof']
    if pchprof == "R":
            Context = "vsright"
    else:
            Context = "vsleft"

    batter = play.find('batter')
    if batter is not None:
        action = batter.attrib.get('action', 'N/A')
    else:
        action = 'N/A'

    # Computing BUNTS
    valid_bunt_actions = re.compile(r".*BU.*", re.IGNORECASE)       #"1B BU", "P2 BU", "23 BUNT","1B BUNT 46","13 BU", "1-3 BU","1B 1 BU","13 BU SAC", "1B 5 BU" , "23 BU" ,"3U SAC BU RBI", "1B BU RBI" ,"53 SAC BU", "23 SAC BU", "1B BUNT RBI" ,"13 SAC BUNT","53 BU","1B 3L BU", "1B LL BU"
    if valid_bunt_actions.search(action):
        Bunts = 1
    else:
        Bunts = 0

    # Computing BUNT 1B
    valid_bunt_1B_actions = re.compile(r"1B.*BU.*", re.IGNORECASE)       #"1B BU", "1B BUNT 46"
    if valid_bunt_1B_actions.search(action):
        Bunt_1B = 1
    else:
        Bunt_1B = 0

    runner = play.find('runner')
    if runner is not None:
        base = runner.attrib.get('start', 'N/A')
        if base == '3':
            # Now also check if the action involves a bunt
            if valid_bunt_actions.search(action):
                Squeeze = 1
            else:
                Squeeze = 0
        else:
            Squeeze = 0  # If base is not 3, no need to check further
    else:
        Squeeze = 0  # If there's no runner, no need to check further

    # Computing Pop-ups
    valid_Pop_Up_actions = re.compile(r"P.*", re.IGNORECASE)
    if valid_Pop_Up_actions.search(action):
        Pop_Ups = 1
    else:
        Pop_Ups = 0

    pitches = play.find('pitches')
    if pitches is not None:
        text = pitches.attrib.get('text', 'N/A')
    else:
        text = 'N/A'

    # Count the number of pitches
    num_pitches = len(text)

    #Infield Singles INF 1B "1B 1", "1B 2", "1B 3", "1B 4", "1B 5", "1B 6"
    valid_inf1b = re.compile(r"1B [1-6]", re.IGNORECASE)
    if valid_inf1b.search(action):
        INF_1B = 1
    else:
        INF_1B = 0

    #Line drive count "L1", "L2", "L3", "L4", "L5", "L6", "L7", "L8", "L9"
    valid_ld = re.compile(r"L[1-9]", re.IGNORECASE)
    if valid_ld.search(action):
      LD_count = 1
    else:
        LD_count = 0

    #Infield fly ball count "F1", "F2", "F3", "F4", "F5", "F6"
    valid_iffb = re.compile(r"F[1-6]", re.IGNORECASE)
    if valid_iffb.search(action):
      IFFB_count = 1
    else:
        IFFB_count = 0

    play_df.append([player_name, Context, Bunts, Bunt_1B, Squeeze, Pop_Ups, num_pitches, INF_1B, LD_count, IFFB_count])

In [ ]:
play_df_1 = pd.DataFrame(play_df, columns=['player_name', 'Context', 'Bunts', 'Bunt_1B', 'Squeeze', 'Pop_Ups', 'num_pitches', 'INF_1B', 'LD_count', 'IFFB_count'])

# Converting the mentioned columns to numeric
cols_to_convert = ['Bunts', 'Bunt_1B', 'Squeeze', 'Pop_Ups','num_pitches', 'INF_1B', 'LD_count', 'IFFB_count']

for col in cols_to_convert:
    play_df_1[col] = pd.to_numeric(play_df_1[col], errors='coerce')

Aggregating the counts based on players

In [ ]:
play_df_new = play_df_1.groupby(['player_name', 'Context']).agg(
    BUNT=('Bunts', 'sum'),
    Bunt_1B=('Bunt_1B', 'sum'),
    Squeeze=('Squeeze', 'sum'),
    Pop_Ups=('Pop_Ups', 'sum'),
    Num_Pitches = ('num_pitches','sum'),
    INF_1B = ('INF_1B','sum'),
    LD_count = ('LD_count', 'sum'),
    IFFB_count = ('IFFB_count', 'sum')
).reset_index()
play_df_new

,player_name,Context,BUNT,Bunt_1B,Squeeze,Pop_Ups,Num_Pitches,INF_1B,LD_count,IFFB_count
0,A. Orr,vsleft,0,0,0,1,68,0,0,1
1,A. Orr,vsright,0,0,0,8,204,2,0,0
2,B. Clark,vsleft,0,0,0,0,17,0,0,0
3,B. Clark,vsright,0,0,0,0,27,1,0,0
4,B. Heidal,vsleft,0,0,0,2,125,0,2,0
5,B. Heidal,vsright,0,0,0,7,272,3,2,0
6,B. Nigh,vsleft,0,0,0,4,115,2,2,0
7,B. Nigh,vsright,2,2,0,6,315,1,2,2
8,C. Hain,vsleft,0,0,0,0,45,0,0,0
9,C. Hain,vsright,0,0,0,2,69,1,0,2


In [ ]:
play_df_bypos = play_df_1.groupby(['player_name']).agg(
    BUNT=('Bunts', 'sum'),
    Bunt_1B=('Bunt_1B', 'sum'),
    Squeeze=('Squeeze', 'sum'),
    Pop_Ups=('Pop_Ups', 'sum'),
    Num_Pitches = ('num_pitches','sum'),
    INF_1B=('INF_1B', 'sum'),
    LD_count = ('LD_count', 'sum'),
    IFFB_count = ('IFFB_count', 'sum')
).reset_index()
play_df_bypos

,player_name,BUNT,Bunt_1B,Squeeze,Pop_Ups,Num_Pitches,INF_1B,LD_count,IFFB_count
0,A. Orr,0,0,0,9,272,2,0,1
1,B. Clark,0,0,0,0,44,1,0,0
2,B. Heidal,0,0,0,9,397,3,4,0
3,B. Nigh,2,2,0,10,430,3,4,2
4,C. Hain,0,0,0,2,114,1,0,2
5,D. Gaskins,0,0,0,2,116,0,1,0
6,E. Larsen,1,1,0,2,219,0,2,0
7,G. Arseneau,0,0,0,5,155,0,2,1
8,H. Griffith,0,0,0,1,37,0,0,0
9,I. Cleary,3,2,0,3,329,4,0,0


        <play seq="24" outs="2" batter="L. Day" batprof="L" pitcher="LINCHEY" pchprof="R" first="I. Cleary" second="R. Bussey">
          <batter name="L. Day" action="1B RS RBI" out="0" adv="1" tobase="1" ab="1" h="1" rbi="1" wherehit="9" wheredesc="right side"></batter>
          <runner base="1" name="I. Cleary" action="++" out="0" adv="2" tobase="3"></runner>
          <runner base="2" name="R. Bussey" action="++" out="0" adv="2" tobase="4" scored="1" por="LINCHEY"></runner>

Pitch Sequence

In [ ]:
pitch_df = []

for play in play_data:
    player_name = play.attrib['batter']
    batprof = play.attrib['batprof']
    pchprof = play.attrib['pchprof']

    if pchprof == "R":
            BatterSide = "R"
    else:
            Batterside = "L"

    if pchprof == "R":
            Context = "vsright"
    else:
            Context = "vsleft"

    pitches = play.find('pitches')
    if pitches is not None:
        PitchSeq = pitches.attrib.get('text', 'N/A')
    else:
        PitchSeq = 'N/A'

    batter = play.find('batter')
    if batter is not None:
        tobase = batter.attrib.get('tobase', 'N/A')
    else:
        tobase = '0'

    pitch_df.append([player_name, BatterSide, Context, PitchSeq, tobase])

pitch_df_1 = pd.DataFrame(pitch_df, columns=['player_name', 'BatterSide', 'Context', 'PitchSeq', 'tobase'])

# Count occurrences of 'S' in the 'PitchSeq' column for each player
pitch_df_1['swingcount'] = pitch_df_1['PitchSeq'].str.count('S')

# Count occurrences of 's' in the 'PitchSeq' column for each player when the first letter of the sequence is 's'
pitch_df_1['SW1'] = pitch_df_1['PitchSeq'].apply(lambda x: 1 if x.startswith('S') else 0)

# Create a new column indicating whether 'k' is the last letter of the sequence
pitch_df_1['Take1K'] = pitch_df_1['PitchSeq'].apply(lambda x: 1 if x.endswith('K') else 0)

# Create a new column indicating whether 'S' is the last letter of the sequence to count the struck out swinging or Swing-and-miss
pitch_df_1['SW_Last'] = pitch_df_1['PitchSeq'].apply(lambda x: 1 if x.endswith('S') else 0)

# Display the DataFrame with the count of 's' for each player
pitch_df_1

#thinking of also making a column for seq-1 which takes off the last pitch, as well as a column for last which is that last letter

,player_name,BatterSide,Context,PitchSeq,tobase,swingcount,SW1,Take1K,SW_Last
0,R. Bussey,R,vsright,KSBBBK,0,1,0,1,0
1,B. Heidal,R,vsright,SBBSP,1,2,1,0,0
2,I. Cleary,R,vsright,FP,0,0,0,0,0
3,B. Nigh,R,vsright,BSKS,0,2,0,0,1
4,L. Day,R,vsright,P,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
1081,A. Orr,R,vsleft,N/A,0,0,0,0,0
1082,I. Cleary,R,vsleft,KB,0,0,0,0,0
1083,B. Heidal,R,vsleft,N/A,0,0,0,0,0
1084,B. Heidal,R,vsleft,SBBBFP,0,1,1,0,0


In [ ]:
# Sum the counts of 's' for each player
player_s_counts = pitch_df_1.groupby('player_name')['swingcount'].sum().reset_index()

# Sum the 'SW1' column for each player
player_swing_counts = pitch_df_1.groupby('player_name')['SW1'].sum().reset_index()

# Sum the counts of 'k' for each player
player_k_counts = pitch_df_1.groupby('player_name')['Take1K'].sum().reset_index()

# Sum the counts of 'SW_LAST' for each player
player_sw_last_counts = pitch_df_1.groupby('player_name')['SW_Last'].sum().reset_index()

# Merge the three DataFrames on 'player_name'
player_counts = pd.merge(player_s_counts, player_swing_counts, on='player_name', how='outer')
player_counts = pd.merge(player_counts, player_k_counts, on='player_name', how='outer')
player_counts = pd.merge(player_counts, player_sw_last_counts, on='player_name', how='outer')

# Rename the columns
player_counts.columns = ['player_name', 'swingcount', 'SW1', 'Take1K' , 'SW_Last']

player_counts_df = pd.DataFrame(player_counts, columns=['player_name', 'swingcount', 'SW1', 'Take1W' , 'SW_Last'])

# Display the DataFrame
player_counts

,player_name,swingcount,SW1,Take1K,SW_Last
0,A. Orr,20,3,2,8
1,B. Clark,3,0,0,2
2,B. Heidal,28,10,8,8
3,B. Nigh,23,2,4,11
4,C. Hain,13,2,1,3
5,D. Gaskins,6,2,1,3
6,E. Larsen,11,3,3,4
7,G. Arseneau,4,1,3,2
8,H. Griffith,6,2,0,3
9,I. Cleary,22,3,7,7


In [ ]:

runner_df = []

for play in play_data:
    runners = play.findall('runner')
    batter = play.find('batter')
    if batter is not None:
        bat_action = batter.attrib.get('action', 'N/A')
    else:
        bat_action = 'N/A'
    for runner in runners:
        player_name = runner.attrib['name']
        action = runner.attrib['action']
        adv = int(runner.attrib['adv'])
        out = int(runner.attrib['out'])
        base = int(runner.attrib['base'])
        tobase = int(runner.attrib['tobase'])
        sb = int(runner.attrib.get('sb', 0))
        cs = int(runner.attrib.get('cs', 0))

        # Computing 1st-3rd on 1B -  baserunner initially at first base (1st) advanced to third base (3rd) on a single (1B)
        if bat_action.startswith("1B") and action == "++" and adv == 2 and out == 0 and base == 1 and tobase == 3:
            base_1_3_Single = 1
        else:
            base_1_3_Single = 0

        #Computing the # times player was on first base when a single was hit
        if bat_action.startswith("1B") and base == 1:
                c_base_1_Single = 1
        else:
                c_base_1_Single = 0

        # Computing 2nd-H on 1B -  baserunner initially at second base (2nd) advanced home (H) on a single (1B)
        if bat_action.startswith("1B") and action == "++" and adv == 2 and out == 0 and base == 2 and tobase == 4:
                base_2_h_Single = 1
        else:
                base_2_h_Single = 0

        #Computing the # times player was on second base when a double was hit
        if bat_action.startswith("1B") and base == 2:
                c_base_2_Single = 1
        else:
                c_base_2_Single = 0

        # Computing 1st-H on 2B -  baserunner initially at first base (1st) advanced home (H) on a double (2B)
        if bat_action.startswith("2B") and action == "+++" and adv == 3 and out == 0 and base == 1 and tobase == 4:
                base_1_h_Double = 1
        else:
                base_1_h_Double = 0

        #Computing the # times player was on first base when a double was hit
        if bat_action.startswith("2B") and base == 1:
                c_base_1_Double = 1
        else:
                c_base_1_Double = 0

        # Computing SB2
        if action == "SB" and tobase == 2 and sb == 1:
                sb2 = 1
        else:
                sb2 = 0

        # Computing SB3
        if action == "SB" and tobase == 3 and sb == 1:
                sb3 = 1
        else:
                sb3 = 0

        # Computing CS2
        if action.endswith("CS") and base == 2 and tobase == 2 and cs == 1 and out == 1:
                cs2 = 1
        else:
                cs2 = 0

        # Computing CS3
        if action.endswith("CS") and base == 3 and tobase == 3 and cs == 1 and out == 1:
                cs3 = 1
        else:
                cs3 = 0

        # Computing SBA2 and SBA3
        SBA2 = sb2 + cs2
        SBA3 = sb3 + cs3
        runner_df.append([player_name, base_1_3_Single, base_2_h_Single, base_1_h_Double, c_base_1_Single, c_base_2_Single, c_base_1_Double, sb2, sb3, cs2, cs3, SBA2, SBA3 ])


In [ ]:
runner_df_1 = pd.DataFrame(runner_df, columns=['player_name', 'base_1_3_Single', 'base_2_h_Single', 'base_1_h_Double' , 'c_base_1_Single', 'c_base_2_Single', 'c_base_1_Double', 'sb2', 'sb3' , 'cs2', 'cs3', 'SBA2', 'SBA3'])

# Converting the mentioned columns to numeric
cols_to_convert = ['base_1_3_Single', 'base_2_h_Single', 'base_1_h_Double' , 'c_base_1_Single', 'c_base_2_Single', 'c_base_1_Double', 'sb2', 'sb3', 'cs2', 'cs3', 'SBA2', 'SBA3']

for col in cols_to_convert:
    runner_df_1[col] = pd.to_numeric(runner_df_1[col], errors='coerce')

runner_df_new = runner_df_1.groupby(['player_name']).agg(
    base_1_3_Single=('base_1_3_Single', 'sum'),
    base_2_h_Single=('base_2_h_Single', 'sum'),
    base_1_h_Double=('base_1_h_Double', 'sum'),
    c_base_1_Single=('c_base_1_Single','sum') ,
    c_base_2_Single=('c_base_2_Single','sum'),
    c_base_1_Double=('c_base_1_Double','sum'),
    sb2 = ('sb2','sum'),
    sb3 = ('sb3','sum'),
    cs2 = ('cs2','sum'),
    cs3 = ('cs3','sum') ,
    SBA2 = ('SBA2','sum'),
    SBA3 = ('SBA3','sum')
).reset_index()
runner_df_new

,player_name,base_1_3_Single,base_2_h_Single,base_1_h_Double,c_base_1_Single,c_base_2_Single,c_base_1_Double,sb2,sb3,cs2,cs3,SBA2,SBA3
0,A. Orr,0,3,0,3,4,1,1,0,1,0,2,0
1,B. Clark,0,0,0,1,0,0,1,0,0,0,1,0
2,B. Heidal,2,2,1,7,5,2,0,1,1,0,1,1
3,B. Nigh,1,1,1,4,3,3,2,0,1,0,3,0
4,C. Hain,1,2,1,5,2,1,0,0,0,0,0,0
5,D. Gaskins,1,0,0,3,0,0,1,0,0,0,1,0
6,E. Larsen,0,1,0,2,2,0,0,0,0,0,0,0
7,G. Arseneau,1,1,0,2,2,0,2,0,0,0,2,0
8,H. Griffith,0,0,0,1,0,0,0,0,0,0,0,0
9,I. Cleary,2,1,0,5,2,1,0,1,0,0,0,1


Computing GDPr - a rate at which a player grounds into double plays with a runner on first

In [ ]:
gdp_df = []

for play in play_data:
    player_name = play.attrib['batter']
    first = play.attrib.get('first', 'N/A')

    batter = play.find('batter')

    if first != 'N/A' and batter is not None:
        ab_first_loaded = batter.attrib.get('ab', 0)
    else:
        ab_first_loaded = 0

    gdp_df.append([player_name, ab_first_loaded])


gdp_df_1 = pd.DataFrame(gdp_df, columns=['player_name','ab_first_loaded'])

# Converting the mentioned columns to numeric
cols_to_convert = ['ab_first_loaded']

for col in cols_to_convert:
    gdp_df_1[col] = pd.to_numeric(gdp_df_1[col], errors='coerce')

gdp_df_new = gdp_df_1.groupby(['player_name']).agg(
    ab_first_loaded=('ab_first_loaded', 'sum')).reset_index()
gdp_df_new

,player_name,ab_first_loaded
0,A. Orr,18
1,B. Clark,1
2,B. Heidal,27
3,B. Nigh,36
4,C. Hain,10
5,D. Gaskins,12
6,E. Larsen,5
7,G. Arseneau,7
8,H. Griffith,4
9,I. Cleary,16


## Merge the seperate dataframes created above

In [ ]:
merged_df_1 = pd.merge(play_df_bypos, runner_df_new, on='player_name', how='left')
merged_df_2 = pd.merge(merged_df_1,gdp_df_new, on='player_name', how='left')
merged_df_2.fillna(0, inplace=True)

# Merge player_counts with merged_df_2
merged_df_final = pd.merge(merged_df_2, player_counts, on='player_name', how='left')

# Fill missing values with 0
merged_df_final.fillna(0, inplace=True)

# Convert specified columns to integer type
columns_to_convert = ['base_1_3_Single', 'base_2_h_Single', 'base_1_h_Double', 'c_base_1_Single', 'c_base_2_Single', 'c_base_1_Double', 'sb2', 'sb3', 'cs2', 'cs3', 'SBA2', 'SBA3']
merged_df_final[columns_to_convert] = merged_df_final[columns_to_convert].astype(int)

# Display the final merged DataFrame
merged_df_final

,player_name,BUNT,Bunt_1B,Squeeze,Pop_Ups,Num_Pitches,INF_1B,LD_count,IFFB_count,base_1_3_Single,...,sb3,cs2,cs3,SBA2,SBA3,ab_first_loaded,swingcount,SW1,Take1K,SW_Last
0,A. Orr,0,0,0,9,272,2,0,1,0,...,0,1,0,2,0,18,20,3,2,8
1,B. Clark,0,0,0,0,44,1,0,0,0,...,0,0,0,1,0,1,3,0,0,2
2,B. Heidal,0,0,0,9,397,3,4,0,2,...,1,1,0,1,1,27,28,10,8,8
3,B. Nigh,2,2,0,10,430,3,4,2,1,...,0,1,0,3,0,36,23,2,4,11
4,C. Hain,0,0,0,2,114,1,0,2,1,...,0,0,0,0,0,10,13,2,1,3
5,D. Gaskins,0,0,0,2,116,0,1,0,1,...,0,0,0,1,0,12,6,2,1,3
6,E. Larsen,1,1,0,2,219,0,2,0,0,...,0,0,0,0,0,5,11,3,3,4
7,G. Arseneau,0,0,0,5,155,0,2,1,1,...,0,0,0,2,0,7,4,1,3,2
8,H. Griffith,0,0,0,1,37,0,0,0,0,...,0,0,0,0,0,4,6,2,0,3
9,I. Cleary,3,2,0,3,329,4,0,0,2,...,1,0,0,0,1,16,22,3,7,7


In [ ]:
#Final_bypos['player_name'] = Final_bypos['Player Name'].apply(lambda x: x.split(',')[1].strip().split()[0][0] + '. ' + x.split(',')[0])
Final_bypos['player_name'] = Final_bypos['Player Name'].apply(lambda x: x.split(',')[0].split()[0][0] + '. ' + x.split(',')[1].strip())

merged_df_3 = pd.merge(Final_bypos, merged_df_final, on='player_name', how='left')
merged_df_3
# Drop the 'player_name' column
merged_df_3 = merged_df_3.drop('player_name', axis=1)

## Convert the dataframe to CSV

In [ ]:
# Save the DataFrame to a CSV file
merged_df_3.to_csv('stats.csv', index=False)
print('XML file has been converted to CSV and saved as stats.csv.')

XML file has been converted to CSV and saved as stats.csv.
